In [1]:
import json

import urllib.request
import urllib.parse
import pandas as pd
import numpy as np
import time
from datetime import datetime
import re
import warnings
import time
from yelpapi import YelpAPI


# import folium
# from folium.plugins import StripePattern
# import seaborn as sns

# import plotly.express as px
# import plotly.io as pio
# # Set this parameter to speed up rendering
# pio.renderers.default = 'iframe'

pd.set_option('display.float_format',lambda x:'%.2f'%x)


In [2]:
pd.set_option('display.max_columns',None)

In [3]:
# !pip install yelpapi

In [4]:
# import zipcodes of interest (ie. Hudson County)
hudson = pd.DataFrame(data = ['07097','07302','07303','07304','07305','07306','07307','07308','07309','07310','07311'], columns = ['Zipcode'])

In [5]:
hudson

,Zipcode
0,07097
1,07302
2,07303
3,07304
4,07305
5,07306
6,07307
7,07308
8,07309
9,07310


In [6]:
# instantiate yelp api object
api_key = 'dPHqb2moIRl9jycy6s82cv8V24GLlxOo9JR6rs0RXcm1JidEuPYn49W8ZuvhMfu_u47Ut-BsoXJqGKnvqyRvz1LXt64VPxKLDen9cD8MCr8EKXqgVVeplN4korj6YXYx'
yelp_api = YelpAPI(api_key)


In [68]:
cat

,alias,title,parents,country_whitelist,country_blacklist
0,3dprinting,3D Printing,localservices,NaN,NaN
1,abruzzese,Abruzzese,italian,[IT],NaN
2,absinthebars,Absinthe Bars,bars,[CZ],NaN
3,acaibowls,Acai Bowls,food,NaN,"[IT, CL, PL, AR, MX, TR]"
4,accessories,Accessories,fashion,NaN,NaN
...,...,...,...,...,...
1560,yugoslav,Yugoslav,restaurants,"[AU, IT, SE, BE, PT, FR]",NaN
1561,zapiekanka,Zapiekanka,food,[PL],NaN
1562,zipline,Ziplining,active,NaN,NaN
1563,zoos,Zoos,active,NaN,NaN


In [67]:
# import categories
cat = pd.read_json('categories.json')
cat = cat.explode('parents')

# petservices
petservices = list(cat[((cat['parents']=='petservices')|(cat['alias']=='petservices'))]['alias'])
# beauty services
spas = list(cat[((cat['parents']=='beautysvc')|(cat['alias']=='beautysvc'))]['alias'])
#all services
services = ['grocery','restaurants','bars','cafes']#+petservices+spas

In [188]:
services

['grocery', 'restaurants', 'bars', 'cafes']

In [7]:
# create df to hold returned results for businesses and their reviews
business_cols = ['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'price', 'phone',
       'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address','zipcode_search','term']
rev_cols = ['id', 'url', 'text', 'rating', 'time_created', 'user.id','user.profile_url', 'user.image_url', 'user.name','business_id']
business_df = pd.DataFrame(columns = business_cols)
review_df = pd.DataFrame(columns = rev_cols)

In [198]:
int(np.round(193/50,0)*50)

200.0

In [200]:

# request business data for each zipcode and save to df
for i in hudson['Zipcode']:
    # example for businesses
    for term in services:
        print("term: "+str(term))
        counter = 0
        offset = 0
        while offset <= counter:
            print("offset: "+str(offset))
            search_results = yelp_api.search_query(term=term,location=i, limit=50,offset=offset)
            print("total: "+str(search_results['total']))

            bus_df = pd.json_normalize(search_results['businesses'])
            # add search terms to returned results
            bus_df['zipcode_search'] = i
            bus_df['term'] = term
            # append returned result to main df
            business_df = business_df.append(bus_df)
            counter = int(np.round(search_results['total']/50,0)*50)
            offset +=50 
business_df = business_df.drop_duplicates('id')

term: grocery
offset: 0
total: 193
offset: 50
total: 193
offset: 100
total: 193
offset: 150
total: 193
offset: 200
total: 193
term: restaurants
offset: 0
total: 250
offset: 50
total: 250
offset: 100
total: 250
offset: 150
total: 250
offset: 200
total: 250
offset: 250
total: 250
term: bars
offset: 0
total: 200
offset: 50
total: 200
offset: 100
total: 200
offset: 150
total: 200
offset: 200
total: 200
term: cafes
offset: 0
total: 232
offset: 50
total: 232
offset: 100
total: 232
offset: 150
total: 232
offset: 200
total: 232
offset: 250
total: 232
term: grocery
offset: 0
total: 372
offset: 50
total: 372
offset: 100
total: 372
offset: 150
total: 372
offset: 200
total: 372
offset: 250
total: 372
offset: 300
total: 372
offset: 350
total: 372
term: restaurants
offset: 0
total: 641
offset: 50
total: 641
offset: 100
total: 641
offset: 150
total: 641
offset: 200
total: 641
offset: 250
total: 641
offset: 300
total: 641
offset: 350
total: 641
offset: 400
total: 641
offset: 450
total: 641
offset: 500

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [ ]:
business_df = business_df[business_df['distance']<5000]

In [9]:
# business_df.to_csv('businesses.csv',encoding = 'utf-8')
business_df = pd.read_csv('businesses.csv',encoding = 'utf-8')

In [52]:
review_df.shape

(3736, 10)

In [48]:
# when code errored out due to too many records per business, pulled the remaining businesses and started scraping again
bus = business_df[~business_df['id'].isin(list(review_df['business_id'].unique()))]

In [56]:
business_df['id'].nunique()

1312

In [47]:
for r in bus['id'].unique():
# for r in business_df['id'].unique():
    counter = 0
    offset = 0
    while offset <= counter:
        print("offset: "+str(offset))
        review = yelp_api.reviews_query(id=r,limit=50,offset=offset)
        # Add time delay to avoid Query per second error
        time.sleep(1)
        print("total: "+str(review['total']))
        
        rev_df = pd.json_normalize(review['reviews'])
        rev_df['business_id'] = r
        review_df = review_df.append(rev_df)
        counter = int(np.round(review['total']/50,0)*50)
        offset +=50

review_df = review_df.drop_duplicates('id')

offset: 0
total: 1
offset: 0
total: 1


In [51]:
review_df = review_df.drop_duplicates('id')

In [63]:
business_df[business_df['review_count']>1000]

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,display_phone,distance,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,zipcode_search,term
539,23,Wm6vj0ekRAzZnaeaBEkOag,taqueria-downtown-jersey-city,Taqueria Downtown,https://s3-media3.fl.yelpcdn.com/bphoto/MOTABi...,False,https://www.yelp.com/biz/taqueria-downtown-jer...,1881,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.00,"['delivery', 'pickup']",$$,12013333220.00,(201) 333-3220,590.37,40.72,-74.04,236 Grove St,NaN,NaN,Jersey City,7302,US,NJ,"['236 Grove St', 'Jersey City, NJ 07302']",7302,restaurants
547,37,FogLdooqSzKbmkwMGSwDiQ,razza-pizza-artigianale-jersey-city-2,Razza Pizza Artigianale,https://s3-media2.fl.yelpcdn.com/bphoto/4FDidt...,False,https://www.yelp.com/biz/razza-pizza-artigiana...,1014,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.00,['delivery'],$$,12013569348.00,(201) 356-9348,496.92,40.72,-74.04,275 Grove St,NaN,NaN,Jersey City,7302,US,NJ,"['275 Grove St', 'Jersey City, NJ 07302']",7302,restaurants
549,41,4ttqjZMCsfz6X0kQ_KR0xQ,orale-mexican-kitchen-jersey-city,Orale Mexican Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/npTbZr...,False,https://www.yelp.com/biz/orale-mexican-kitchen...,1224,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.00,['delivery'],$$,12013330001.00,(201) 333-0001,497.76,40.72,-74.04,341 Grove St,NaN,NaN,Jersey City,7302,US,NJ,"['341 Grove St', 'Jersey City, NJ 07302']",7302,restaurants
672,15,s-UpwopAqVV7hbLH2x7kDg,and-plus-salon-new-york,& PLUS Salon,https://s3-media3.fl.yelpcdn.com/bphoto/TerrE4...,False,https://www.yelp.com/biz/and-plus-salon-new-yo...,1173,"[{'alias': 'hair', 'title': 'Hair Salons'}]",4.50,[],$$,12127777718.00,(212) 777-7718,3892.27,40.72,-74.00,41 Wooster St,NaN,NaN,New York,10012,US,NY,"['41 Wooster St', 'New York, NY 10012']",7302,hair
985,38,pL2tig3FxJcrZ4wDIO4TsA,eataly-nyc-downtown-new-york-2,Eataly NYC Downtown,https://s3-media2.fl.yelpcdn.com/bphoto/VMUIz7...,False,https://www.yelp.com/biz/eataly-nyc-downtown-n...,1068,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.50,"['delivery', 'pickup']",$$,12128972895.00,(212) 897-2895,2637.68,40.71,-74.01,101 Liberty St,Fl 3,NaN,New York,10006,US,NY,"['101 Liberty St', 'Fl 3', 'New York, NY 10006']",7308,grocery
1002,14,RemvTpxalDjSew5HwmNzUQ,the-dead-rabbit-new-york,The Dead Rabbit,https://s3-media2.fl.yelpcdn.com/bphoto/_4T5R0...,False,https://www.yelp.com/biz/the-dead-rabbit-new-y...,1649,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.00,"['delivery', 'pickup']",$$,16464227906.00,(646) 422-7906,3060.88,40.70,-74.01,30 Water St,NaN,NaN,New York,10004,US,NY,"['30 Water St', 'New York, NY 10004']",7308,bars
1186,47,J4pAppaiXwCkbLOGc4wHww,raines-law-room-new-york,Raines Law Room,https://s3-media1.fl.yelpcdn.com/bphoto/1MDwRO...,False,https://www.yelp.com/biz/raines-law-room-new-y...,1502,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",4.50,['delivery'],$$$,nan,NaN,3769.27,40.74,-73.99,48 W 17th St,NaN,NaN,New York,10011,US,NY,"['48 W 17th St', 'New York, NY 10011']",7310,bars


In [70]:
review_df.shape

(3736, 12)

In [62]:
review_df['date'] = pd.to_datetime(review_df['time_created'])
review_df['year'] = review_df['date'].dt.year

In [64]:
review_df['rating'] = review_df['rating'].astype(int) 

In [71]:
review_df.to_csv('reviews.csv',encoding = 'utf-8')

In [180]:
ratings_year = review_df.groupby(['business_id','year'])['rating'].agg(['size','mean']).reset_index().sort_values(by=['business_id','year'])  


In [181]:
len(ratings_year['business_id'].unique())

804

In [102]:
ratings_year['year'].min()

2011

In [106]:
business_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,display_phone,distance,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,zipcode_search,term
0,gNtfNKjHBMi3O5lf8uquUg,stop-and-shop-jersey-city,Stop & Shop,https://s3-media1.fl.yelpcdn.com/bphoto/qgsp7M...,False,https://www.yelp.com/biz/stop-and-shop-jersey-...,48,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.00,[],$$,+12016536508,(201) 653-6508,1770.42,40.74,-74.05,232 Central Ave,,,Jersey City,07306,US,NJ,"[232 Central Ave, Jersey City, NJ 07306]",07097,grocery
1,fKWHGkJkVrTZ1_yCyNlXwA,trader-joes-hoboken-2,Trader Joe's,https://s3-media3.fl.yelpcdn.com/bphoto/Xv1o2V...,False,https://www.yelp.com/biz/trader-joes-hoboken-2...,106,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.00,[],$$,+12017144790,(201) 714-4790,3202.04,40.75,-74.03,1350 Willow Ave,,None,Hoboken,07030,US,NJ,"[1350 Willow Ave, Hoboken, NJ 07030]",07097,grocery
2,t5332TeSXTn5TBOY4T_lOA,shoprite-of-hoboken-hoboken,ShopRite of Hoboken,https://s3-media2.fl.yelpcdn.com/bphoto/MCuij0...,False,https://www.yelp.com/biz/shoprite-of-hoboken-h...,142,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.50,[],$$,+12017926070,(201) 792-6070,2693.57,40.75,-74.04,900 Madison Ave,,,Hoboken,07030,US,NJ,"[900 Madison Ave, Hoboken, NJ 07030]",07097,grocery
3,67sm-6hffaXPsqyA8CXvtg,youngs-farm-jersey-city,Young's Farm,https://s3-media2.fl.yelpcdn.com/bphoto/SU4au7...,False,https://www.yelp.com/biz/youngs-farm-jersey-ci...,8,"[{'alias': 'markets', 'title': 'Fruits & Veggi...",4.00,[],$,+12012227788,(201) 222-7788,1749.54,40.74,-74.05,294 Central Ave,,,Jersey City,07307,US,NJ,"[294 Central Ave, Jersey City, NJ 07307]",07097,grocery
4,3t_WBe_6fPFFObg6lt8UIg,p-and-k-food-market-jersey-city-2,P & K Food Market,https://s3-media2.fl.yelpcdn.com/bphoto/EhAVIk...,False,https://www.yelp.com/biz/p-and-k-food-market-j...,75,"[{'alias': 'grocery', 'title': 'Grocery'}]",5.00,[],$$,+12014519888,(201) 451-9888,3982.64,40.72,-74.04,170 Newark Ave,,,Jersey City,07302,US,NJ,"[170 Newark Ave, Jersey City, NJ 07302]",07097,grocery
